In [1]:
import gzip
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv('chicago-complete.weekly.2019-02-04-to-2019-02-10/data.csv.gz', compression='gzip')

In [3]:
df.tail()

,timestamp,node_id,subsystem,sensor,parameter,value_raw,value_hrf
28330972,2019/02/10 23:59:58,001e0610f703,metsense,pr103j2,temperature,581,-1.8
28330973,2019/02/10 23:59:58,001e0610f703,metsense,spv1840lr5h_b,intensity,NaN,62.74
28330974,2019/02/10 23:59:58,001e0610f703,metsense,tmp112,temperature,65425,-254.81
28330975,2019/02/10 23:59:58,001e0610f703,metsense,tsl250rd,intensity,7,0.352
28330976,2019/02/10 23:59:58,001e0610f703,metsense,tsys01,temperature,9043376,-1.38


In [4]:
df.describe()

,timestamp,node_id,subsystem,sensor,parameter,value_raw,value_hrf
count,28330977,28330977,28330977,28330977,28330977,26181597,26340186
unique,469280,41,8,54,59,861507,868629
top,2019/02/07 18:16:33,001e06113107,metsense,mma8452q,temperature,65535,-0.977
freq,336,1084474,11724097,2238159,7952779,3771481,671515


In [5]:
df.shape

(28330977, 7)

In [7]:
sensors = pd.read_csv("chicago-complete.weekly.2019-02-04-to-2019-02-10/sensors.csv")

### Filtering out based on required parameters and attributes.

In [8]:
df['parameter'].unique()

array(['alphasense', 'coresense', 'modem', 'wagman', 'load_1', 'load_10',
       'load_5', 'free', 'total', 'rx', 'tx', 'state', 'substate',
       'uptime', 'idletime', 'intensity', 'humidity', 'temperature',
       'pressure', 'id', 'magnetic_field_x', 'magnetic_field_y',
       'magnetic_field_z', 'acceleration_x', 'acceleration_y',
       'acceleration_z', 'concentration', 'ir_intensity', 'uv_intensity',
       'visible_light_intensity', 'fw', 'bins', 'pm1', 'pm10', 'pm2_5',
       'sample_flow_rate', 'sampling_period', 'bottom_camera',
       'microphone', 'top_camera', 'used', 'current', 'other', 'count',
       'flags', 'cs', 'ep', 'nc', 'commit', 'major', 'minor', 'patch',
       'battery', 'brainplate', 'ep_heatsink', 'nc_heatsink',
       'powersupply', 'car_total', 'person_total'], dtype=object)

In [9]:
req_para = ['intensity', 'humidity', 'temperature',
       'pressure', 'magnetic_field_x', 'magnetic_field_y',
       'magnetic_field_z', 'acceleration_x', 'acceleration_y',
       'acceleration_z', 'concentration', 'ir_intensity', 'uv_intensity',
       'visible_light_intensity', 'pm1', 'pm10', 'pm2_5',
       'car_total', 'person_total']

In [10]:
df = df[df['parameter'].isin(req_para)]

In [11]:
df.drop(columns = 'value_raw', inplace = True)

In [ ]:
df['sensor'].unique()

In [13]:
sensors = sensors[sensors['sensor'].isin(list(df['sensor'].unique()))]
sensors = sensors[sensors['parameter'].isin(list(df['parameter'].unique()))]

In [14]:
df.head()

,timestamp,node_id,subsystem,sensor,parameter,value_hrf
18,2019/02/04 00:00:01,001e0610e532,lightsense,apds_9006_020,intensity,0.241
19,2019/02/04 00:00:01,001e0610e532,lightsense,hih6130,humidity,41.09
20,2019/02/04 00:00:01,001e0610e532,lightsense,hih6130,temperature,26.19
21,2019/02/04 00:00:01,001e0610e532,lightsense,ml8511,intensity,43.638
22,2019/02/04 00:00:01,001e0610e532,lightsense,mlx75305,intensity,2.032


In [15]:
sensors.drop(columns = ['ontology', 'datasheet'], inplace = True)

### Data Cleaning

In [16]:
# Fix nulls in sensor data
sensors['hrf_maxval'] = sensors.hrf_maxval.replace(np.NaN, '100000')
sensors['hrf_minval'] = sensors.hrf_minval.replace(np.NaN, '0.0')

In [17]:
df = pd.merge(df, sensors, left_on =['sensor', 'parameter', 'subsystem'], right_on =['sensor', 'parameter', 'subsystem'], how = 'left')

In [18]:
# Drop rows where sensors have not picked up a reading
df.dropna(axis = 0, subset = ['value_hrf'], inplace = True)

In [19]:
# Image subsystem ranges and unit fixed
df['hrf_maxval'] = df.hrf_maxval.replace(np.NaN, '100000')
df['hrf_minval'] = df.hrf_minval.replace(np.NaN, '0')
df['hrf_unit'] = df.hrf_unit.replace(np.NaN, 'count')

In [20]:
# No nulls
df.isna().sum()

timestamp     0
node_id       0
subsystem     0
sensor        0
parameter     0
value_hrf     0
hrf_unit      0
hrf_minval    0
hrf_maxval    0
dtype: int64

In [21]:
df.head()

,timestamp,node_id,subsystem,sensor,parameter,value_hrf,hrf_unit,hrf_minval,hrf_maxval
0,2019/02/04 00:00:01,001e0610e532,lightsense,apds_9006_020,intensity,0.241,lux,0,1000
1,2019/02/04 00:00:01,001e0610e532,lightsense,hih6130,humidity,41.09,RH,0,100
2,2019/02/04 00:00:01,001e0610e532,lightsense,hih6130,temperature,26.19,C,-25,85
3,2019/02/04 00:00:01,001e0610e532,lightsense,ml8511,intensity,43.638,uW/cm^2,0,15
4,2019/02/04 00:00:01,001e0610e532,lightsense,mlx75305,intensity,2.032,uW/cm^2,0,160


In [22]:
def range_checker(row):
    if float(row['hrf_minval']) <= float(row['value_hrf']) <= float(row['hrf_maxval']):
        return row['value_hrf']
    else: return np.nan

In [24]:
df['value_hrf'] = df.apply(lambda x: range_checker(x), axis = 1)

In [25]:
df.isna().sum()

timestamp           0
node_id             0
subsystem           0
sensor              0
parameter           0
value_hrf     5169678
hrf_unit            0
hrf_minval          0
hrf_maxval          0
dtype: int64

In [26]:
df.head()

,timestamp,node_id,subsystem,sensor,parameter,value_hrf,hrf_unit,hrf_minval,hrf_maxval
0,2019/02/04 00:00:01,001e0610e532,lightsense,apds_9006_020,intensity,0.241,lux,0,1000
1,2019/02/04 00:00:01,001e0610e532,lightsense,hih6130,humidity,41.09,RH,0,100
2,2019/02/04 00:00:01,001e0610e532,lightsense,hih6130,temperature,26.19,C,-25,85
3,2019/02/04 00:00:01,001e0610e532,lightsense,ml8511,intensity,NaN,uW/cm^2,0,15
4,2019/02/04 00:00:01,001e0610e532,lightsense,mlx75305,intensity,2.032,uW/cm^2,0,160


In [27]:
df.dropna(axis = 0, subset = ['value_hrf'], inplace = True)

In [28]:
df.to_csv('data_cleaned.csv', index = False)